In [1]:
import torch
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader

DATA_DIR = "../../data/"

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_products.csv",
                   source_column="link",
                   metadata_columns=["id", "side_menu_text", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["id", "content", "side_menu_text", "link"],
                       }
                   )
product_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}faq_final.csv",
                  source_column="id",
                  metadata_columns=["context", "question"],
                  content_columns=["answer"],
                  csv_args={
                        "fieldnames": ["id", "context", "question", "answer"],
                        }
                  )

data = loader.load()[1:]
print(data[0])
print(product_data[0])

page_content='answer: Цалингийн зээлийг сүүлийн 6 сар ажил эрхэлсэн, үндсэн ажилтнаар ажиллаж буй гэрээт байгууллагын ажилтанд олгоно. Одоо ажиллаж буй байгууллагадаа шилжин ажиллахдаа үндсэн ажилтнаар орсон бол цалингийн зээл авч болно.' metadata={'source': 'faq_10_0', 'row': 1, 'context': 'Цалингийн зээл', 'question': 'Цалингийн зээл авахад ямар шалгуур тавих вэ?'}
page_content='content: Цалингийн зээл
Харилцагч та цалингаа ХасБанкаар дамжуулан авдаг бол “Цалингийн зээл” авч өөрийн тогтмол орлогоор өөртөө болон гэр бүлдээ хөрөнгө оруулах боломжтой.
Давуу тал:
Хурдан шуурхай шийдвэрлэн олгодог;
Зээлийн хугацаанд нэмэлт санхүүжилтээр зээлийн хэмжээг нэмэгдүүлдэг.
Бүтээгдэхүүний нөхцөл
нөхцөл
дижитал банкаар
салбараар
Зээлийн хэмжээ
300,000 - 50,000,000 төгрөг
300,000 төгрөгөөс дээш
Зээлийн хүү*
Сарын
1.5%-1.9%
1.5% - 1.8%
Жилийн
18.0%-22.8%
18.0% - 21.6%**
Зээлийн хугацаа
30 сар
Үйлчилгээний шимтгэл
0.5%
1%
* - ХасБанктай гэрээтэй байгууллагын ажилтан бол гэрээгээр тохирсон хүүг мөрдөн

In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=26, chunk_overlap=4
)
docs = text_splitter.split_documents(documents=data)
print(docs[5])

page_content='answer: Гүйлгээ нь интернэт банкны гүйлгээ интернэт банкны гүйлгээний шимтгэл хураамж тооцогдоно.Хэрвээ ХасБанкны харилцагч ХасБанкны үүсгэсэн эсхүл ХасБанкинд данстай бусдын үүсгэсэн мерчант руу гүйлгээ хийхэд гүйлгээний шимтгэл үгүй.ХасБанкны харилцагч бусад банкны үүсгэсэн эсхүл бусад банкны дансаа мерчантын дансаар сонгосон харилцагч руу гүйлгээ хийхэд банк хоорондын гүйлгээний шимтгэл тооцогдоно.' metadata={'source': 'faq_13_4', 'row': 6, 'context': 'QPay үйлчилгээ', 'question': 'QR ашиглан төлөхөд харилцагчийн төлөх шимтгэл хураамж:'}


In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [6]:
vector_store = FAISS.from_documents(docs, embeddings)
vector_store.save_local("faiss_index")

In [7]:
vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [8]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# docs = retriever.invoke("Qpay үйлчилгээ гэж юу юэ")
docs = retriever.invoke("QPay үйлчилгээ гэж юу вэ?")
docs

[Document(metadata={'source': 'faq_13_0', 'row': 2, 'context': 'QPay үйлчилгээ', 'question': '‘QPay үйлчилгээ’ гэж юу вэ?'}, page_content='answer: ХасБанкны интернэт банкны харилцагч ухаалаг утсанд зориулсан “XacBank” аппликейшныг ашиглан QR /КЮ-АР/ кодыг уншуулан төлбөр төлөх, өөрийн дансандаа QR код үүсгэх буюу нэхэмжлэх үүсгэж дансандаа орлого хүлээн авах боломжтой үйлчилгээ юм.'),
 Document(metadata={'source': 'faq_13?lang=mn_0', 'row': 7, 'context': 'QPay үйлчилгээ', 'question': '‘QPay үйлчилгээ’ гэж юу вэ?'}, page_content='answer: ХасБанкны интернэт банкны харилцагч ухаалаг утсанд зориулсан “XacBank” аппликейшныг ашиглан QR /КЮ-АР/ кодыг уншуулан төлбөр төлөх, өөрийн дансандаа QR код үүсгэх буюу нэхэмжлэх үүсгэж дансандаа орлого хүлээн авах боломжтой үйлчилгээ юм.'),
 Document(metadata={'source': 'faq_13_2', 'row': 4, 'context': 'QPay үйлчилгээ', 'question': 'Хэн ашиглах боломжтой байх вэ?'}, page_content='answer: QR унших боломж нь ХасБанк АПП дээр шинэ нэмэлт функц болж нэмэгдэ

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from transformers import AutoModelForMaskedLM

model_id = "meta-llama/Llama-3.1-8B"

# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
# model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16,
    device_map='auto'
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # llm-н үүсгэж болох токений дээд хязгаар
    max_new_tokens=1024,
    # max_new_tokens=32,
    # хариултын randomization-г арилгах
    do_sample=True,
    top_k=1,
    repetition_penalty=1.15,
    # гаралт бидний өгсөн prompt-г хамт хэвлэхгүй байх
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id,
)

# HuggingFace pipeline-г LangChain-ы pipeline болгох
llm = HuggingFacePipeline(pipeline=pipe)

/root/rag-starter/.virt/lib/python3.10/site-packages/accelerate/utils/modeling.py:1590: UserWarning: Current model requires 4224 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


In [12]:
# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the user question. If you don't know the answer to the question, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     'Context:\n"""\n{context}\n"""'
# )

lang = "Mongolian"

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the user question. If you don't know the answer to the question, say that you "
    "don't know. The answer should be given in Mongolian."
    "Keep the answer concise"
    "\n\n"
    'Context:\n"""\n{context}\n"""'
)

# system_prompt = (
#     """
#         Your are a helpful AI assistant for question-answering tasks. Use the o
#     """
# )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Question: \"\"\"{input}\"\"\""),
        ("assistant", "Answer: "),
    ]
)

# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

docs = retriever.invoke("QPay үйлчилгээ гэж юу вэ?")
print(docs[0])
# Chain үүсгэх
# input -> retrieval -> prompt -> llm -> answer
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

page_content='answer: ХасБанкны интернэт банкны харилцагч ухаалаг утсанд зориулсан “XacBank” аппликейшныг ашиглан QR /КЮ-АР/ кодыг уншуулан төлбөр төлөх, өөрийн дансандаа QR код үүсгэх буюу нэхэмжлэх үүсгэж дансандаа орлого хүлээн авах боломжтой үйлчилгээ юм.' metadata={'source': 'faq_13_0', 'row': 2, 'context': 'QPay үйлчилгээ', 'question': '‘QPay үйлчилгээ’ гэж юу вэ?'}


In [ ]:
# response = rag_chain.invoke({"input": "Qpay үйлчилгээ гэж юу вэ?"})
# response = rag_chain.invoke({"input": "дебит карт яаж авах вэ"})
response = rag_chain.invoke({"input": "What is 1+1"})
print(response)

response["answer"]